In [1]:
pip install markovify

     |████████████████████████████████| 245kB 6.9MB/s 
  Created wheel for markovify: filename=markovify-0.9.0-cp36-none-any.whl size=18475 sha256=67d70d25abab30308cdccdc90a8e7edbe21a0bb638f3dd87ffbc39ca0f1f90bc
  Stored in directory: /root/.cache/pip/wheels/03/59/5b/04c5f27d57977580c1122fa69a69c327dede5a30f9dac54c55
Successfully built markovify


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import nltk
import spacy
import markovify

import warnings
warnings.filterwarnings(action="ignore")

!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [4]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'twitter_sentiment'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

tweets_df = pd.read_sql_query('select * from twitter',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


In [6]:
tweets_df.head()

,index,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,570306133677760513,neutral,1.0000,None,NaN,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
1,1,570301130888122368,positive,0.3486,None,0.0000,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
2,2,570301083672813571,neutral,0.6837,None,NaN,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,@VirginAmerica it's really aggressive to blast...,None,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
4,4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)


### 1. First, do some data preprocessing to clean up the data as necessary. You can use your solution to the assignment of the Text preprocessing checkpoint.


In [8]:
nlp = spacy.load('en')

# below is necessary to avoid memory errors with spacy
nlp.max_length = 20000000

# all the processing work is done below, so it may take a while 

twitter_doc = nlp(" ".join(tweets_df.text))

### Train a Markov chain model by using only the negative sentiment tweets. Generate some random sentences. Do the generated sentences exhibit the same negative sentiment?

In [9]:
twitter_negs = tweets_df.loc[tweets_df.airline_sentiment == 'negative'].text
twitter_negs

3        @VirginAmerica it's really aggressive to blast...
4        @VirginAmerica and it's a really big bad thing...
5        @VirginAmerica seriously would pay $30 a fligh...
15           @VirginAmerica SFO-PDX schedule is still MIA.
17       @VirginAmerica  I flew from NYC to SFO last we...
                               ...                        
14631    @AmericanAir How do I change my flight if the ...
14633    @AmericanAir thx for nothing on getting us out...
14635    @AmericanAir my flight was Cancelled Flightled...
14636    @AmericanAir leaving over 20 minutes Late Flig...
14638    @AmericanAir you have my money, you change my ...
Name: text, Length: 9178, dtype: object

In [11]:
# processing work done here

neg_tweets_doc = nlp(" ".join(twitter_negs))

neg_tweets_sents = " ".join([sent.text for sent in neg_tweets_doc.sents if len(sent.text) > 1 ])

In [14]:
neg_tweets_gen = markovify.Text(neg_tweets_sents,state_size = 3)

# three randomly generated sentences 
for i in range(3):
  print(neg_tweets_gen.make_sentence(tries=100))

print('\n'+'-'*40+'\n')

# tree randomly generated sentences of no more than 100 characters

for i in range(3):
  print(neg_tweets_gen.make_short_sentence(100,tries=100))

@SouthwestAir Trying to get my boyfriend booked on the same flight I was put on flight tomorrow AM, arriving home at 4 pm.
@united I'm not sure how you guys don't know where my bags are?
@united after 2 days - for multiple hours on hold got rebooked on on us airways by you guys but the flight I have been loyal to Southwest from the beginning but very unhappy about your devaluation.

----------------------------------------

@JetBlue what's going on with United's Mileage Program?
@SouthwestAir Flight Cancelled Flighted, two hours on the phone who knew what they were doing.
What's up with the wait times on your customer service phone off the hook!


### 3. Repeat the previous task, but this time, use only the positive sentiment tweets. Generate some random sentences and observe whether they exhibit positive sentiment or not.

In [15]:
twitter_pos = tweets_df.loc[tweets_df.airline_sentiment == 'positive'].text
twitter_pos

1        @VirginAmerica plus you've added commercials t...
6        @VirginAmerica yes, nearly every time I fly VX...
8          @virginamerica Well, I didn't…but NOW I DO! :-D
9        @VirginAmerica it was amazing, and arrived an ...
11       @VirginAmerica I &lt;3 pretty graphics. so muc...
                               ...                        
14621    @AmericanAir I love your company and your staf...
14625    @AmericanAir Love the new planes for the JFK-L...
14627    @AmericanAir Flight 236 was great. Fantastic c...
14630    Thank you. “@AmericanAir: @jlhalldc Customer R...
14632                          @AmericanAir Thanks! He is.
Name: text, Length: 2363, dtype: object

In [16]:
# processing work done here

pos_tweets_doc = nlp(" ".join(twitter_pos))

pos_tweets_sents = " ".join([sent.text for sent in pos_tweets_doc.sents if len(sent.text) > 1 ])

In [17]:
pos_tweets_gen = markovify.Text(pos_tweets_sents,state_size = 3)

# three randomly generated sentences 
for i in range(3):
  print(pos_tweets_gen.make_sentence(tries=100))

print('\n'+'-'*40+'\n')

# tree randomly generated sentences of no more than 100 characters

for i in range(3):
  print(pos_tweets_gen.make_short_sentence(100,tries=100))

@SouthwestAir - I just had a great flight #4223 with Damion!
#flyfi http://t.co/8jceDiKY9U @JetBlue thanks @JetBlue Then en route to LHR @united thank you!
@SouthwestAir @DeltaPoints hey at least you guys are and this rolled across.

----------------------------------------

Thank you to the great crew on flight 206 is awesome!
@SouthwestAir lol I already am ! I am a happy customer @united thanx so much.
2 mths waiting @AmericanAir to speak to someone things got fixed very quick.


### 4. Now, train your Markov chain model using all of the tweets. Generate some random sentences. Can you say something systematic about the sentiments of the generated tweets?

In [19]:
twitter_sents = " ".join([sent.text for sent in twitter_doc.sents if len(sent.text) > 1 ])

In [21]:
tweets_gen = markovify.Text(twitter_sents,state_size=3)

# three randomly generated sentences 
for i in range(3):
  print(tweets_gen.make_sentence(tries=100))

print('\n'+'-'*40+'\n')

# tree randomly generated sentences of no more than 100 characters

for i in range(3):
  print(tweets_gen.make_short_sentence(100,tries=100))

#loveisintheair http://t.co/DoCmvoTWTI @JetBlue She claimed that she did, but I was the last drop for me; plan to avoid AA as possible.
@JetBlue some woman stole my seat on the plane now we need to go to work tomorrow am.
@united I took care of it @USAirways I have corporate-paid travel in April.

----------------------------------------

Was able to get on that, please.
I made a reservation online but I didn't make the Flight Booking Problems.
@united: thanks for the show!


We can see both positive and negative tweets this time